In [ ]:
import pandas as pd
import numpy as np
import math
import os
from matplotlib import pyplot as plt
import matplotlib as mpl
import sys
from scipy.interpolate import griddata
from tqdm import tqdm
# sys.path.append('/Users/jiarong/Google Drive/codes/jiarongw-postprocessing/functions')
sys.path.append('/projects/DEIKE/jiarongw/jiarongw-postprocessing/jupyter_notebook/functions/')
from fio import readin
# plt.style.use('/projects/DEIKE/jiarongw/jiarongw-postprocessing/media/matplotlib/stylelib/pof.mplstyle')
plt.style.use('/projects/DEIKE/jiarongw/jiarongw-postprocessing/media/matplotlib/stylelib/jfm.mplstyle')

## Summary:
This notebook analyze the bulk velocity and height etc.
## <a class="anchor" id="0">Table of content: </a> 

#### [1. Define case and read-in functions](#1)
#### [2. Illustration of layers for paper](#2)

In [ ]:
from func import array_to_mesh, read_t

In [ ]:
class Case(object):
    """ This class defines methods specific to cases.
        Attributes: 
            self.NL 
    """
    def __init__(self, NL=20, LEVEL=8, path='/projects/DEIKE/jiarongw/multilayer/stokes/stokes_8_20_Htheta0.51/'):
        self.NL = NL 
        self.LEVEL = LEVEL
        self.path = path    

### 3D reconstruction of the field

In [ ]:
case_field1 = Case(NL=15, LEVEL=10, path='/projects/DEIKE/jiarongw/multilayer/revision/field_new_200m_P0.02_RE40000_10_15_rand4_Htheta0.503/')
case = case_field1
case.L = 200; case.H = 40; case.kp = 2*np.pi/case.L*5; case.cp = (9.8/case.kp)**0.5

def plot_func(t, ax):
    global case
    Nh = 2**case.LEVEL; Nl = case.NL; L = case.L; H = case.H; cp = case.cp; kp = case.kp
    h_ensem, ux_ensem, uy_ensem, uz_ensem = read_t(t=t, Nh=Nh, Nl=Nl, path=case.path)
    # Additional filtering due to wrong outputs, only for some old cases
#     h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]
#     ux_ensem[0,:,:] = ux_ensem[0,:,:]*0; uy_ensem[0,:,:] = uy_ensem[0,:,:]*0; uz_ensem[0,:,:] = uz_ensem[0,:,:]*0
    
    x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh = array_to_mesh (h_ensem, ux_ensem, uy_ensem, uz_ensem, L0=L, H=H, Nh=Nh, Nl=Nl)
    yslice = 128
    image = ax.pcolormesh(x_mesh[:,:,yslice], z_mesh[:,:,yslice], ux_mesh[:,:,yslice]/cp, shading='flat', cmap='RdBu_r', vmax=0.5, vmin=-0.5)
    ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/5,0,L/10])
    ax.set_ylim([-L/5, L/10])
#     ax.spines.right.set_visible(False)
#     ax.spines.top.set_visible(False)
#     ax.axis('off'); 
#     ax.text(0.05, 0.1, '$t=%gT$' %t, transform=ax.transAxes, fontsize=8)
    ''' Plot the surface '''
    xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
    ax.plot(xarray,np.sum(h_ensem[:,:,128],axis=0)-H,lw=0.5,c='k')
    return image, ux_mesh, uy_mesh, uz_mesh

t = 120
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
image, ux_mesh1, uy_mesh1, uz_mesh1 = plot_func(t, ax)

In [ ]:
Nh = 2**case.LEVEL; Nl = case.NL; L = case.L; H = case.H; cp = case.cp; kp = case.kp
x_mesh_t = []
y_mesh_t = []
z_mesh_t = []
ux_mesh_t = []
uy_mesh_t = []
uz_mesh_t = []
for t in tqdm((0, 50, 100, 150)):
    h_ensem, ux_ensem, uy_ensem, uz_ensem = read_t(t=t, Nh=Nh, Nl=Nl, path=case.path)
    x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh = array_to_mesh (h_ensem, ux_ensem, uy_ensem, uz_ensem, L0=L, H=H, Nh=Nh, Nl=Nl)
    x_mesh_t.append(x_mesh); y_mesh_t.append(y_mesh); z_mesh_t.append(z_mesh)
    ux_mesh_t.append(ux_mesh); uy_mesh_t.append(uy_mesh); uz_mesh_t.append(uz_mesh)

In [ ]:
def plot_func(t, ax):
    global case
    Nh = 2**case.LEVEL; Nl = case.NL; L = case.L; H = case.H; cp = case.cp; kp = case.kp
    [h_ensem, ux_ensem, uy_ensem, uz_ensem, omegax_ensem, omegay_ensem, omegaz_ensem] = read_t(fieldnames=['h','ux','uy','uz','omegax','omegay','omegaz'], 
                                                                                 t=t, Nh=Nh, Nl=Nl, path=case.path)
    # Additional filtering due to wrong outputs, only for some old cases
#     h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]
#     ux_ensem[0,:,:] = ux_ensem[0,:,:]*0; uy_ensem[0,:,:] = uy_ensem[0,:,:]*0; uz_ensem[0,:,:] = uz_ensem[0,:,:]*0
    
    x_mesh, y_mesh, z_mesh = array_to_mesh (h_ensem, L0=L, H=H, Nh=Nh, Nl=Nl)
    yslice = 128
    image = ax.pcolormesh(x_mesh[:,:,yslice], z_mesh[:,:,yslice], omegax_ensem[:,:,yslice], shading='flat', cmap='RdBu_r')
    ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/5,0,L/10])
    ax.set_ylim([-L/5, L/10])
#     ax.spines.right.set_visible(False)
#     ax.spines.top.set_visible(False)
#     ax.axis('off'); 
#     ax.text(0.05, 0.1, '$t=%gT$' %t, transform=ax.transAxes, fontsize=8)
    ''' Plot the surface '''
    xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
    ax.plot(xarray,np.sum(h_ensem[:,:,128],axis=0)-H,lw=0.5,c='k')
    return image, x_mesh, y_mesh, z_mesh, ux_ensem, uy_ensem, uz_ensem, omegax_ensem, omegay_ensem, omegaz_ensem

In [ ]:
# case_field1 = Case(NL=15, LEVEL=10, path='/projects/DEIKE/jiarongw/multilayer/revision/field_new_200m_P0.005_RE40000_10_15_rand2_Htheta0.503/vort/')
# case = case_field1
# case.L = 200; case.H = 40; case.kp = 2*np.pi/case.L*5; case.cp = (9.8/case.kp)**0.5
# t = 400
# fig = plt.figure (figsize=[4,2]); ax = plt.gca()
# image, x_mesh1, y_mesh1, z_mesh1, ux_mesh1, uy_mesh1, uz_mesh1, omegax_mesh1, omegay_mesh1, omegaz_mesh1 = plot_func(t, ax)

case_field2 = Case(NL=15, LEVEL=10, path='/projects/DEIKE/jiarongw/multilayer/revision/field_new_200m_P0.02_RE40000_10_15_rand4_Htheta0.503/vort/')
case = case_field2
case.L = 200; case.H = 40; case.kp = 2*np.pi/case.L*5; case.cp = (9.8/case.kp)**0.5
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
t = 0
image, x_mesh2, y_mesh2, z_mesh2, ux_mesh2, uy_mesh2, uz_mesh2, omegax_mesh2, omegay_mesh2, omegaz_mesh2 = plot_func(t, ax)

In [ ]:
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
yslice = 80; L = 200; Nh = 1024

ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/5,0,L/10])
ax.set_ylim([-L/5, L/10])
# xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
# ax.plot(xarray,z_mesh[-1,1:,128],lw=0.5,c='k')

image = plt.pcolormesh(x_mesh1[:,:,yslice], z_mesh1[:,:,yslice], omegay_mesh1[:,:,yslice], shading='flat', 
                       cmap='RdBu_r', vmax=0.4, vmin=-0.4)
# image = plt.pcolormesh(x_mesh2[:,:,yslice], z_mesh2[:,:,yslice], omegay_mesh2[:,:,yslice], shading='flat', 
#                        cmap='RdBu_r', vmax=0.4, vmin=-0.4)

cbar = plt.colorbar(image, orientation='vertical')
cbar.ax.text(-0.5, 1.05, r'$\omega_y (s^{-1})$', transform=cbar.ax.transAxes, fontsize=8)
cbar.set_ticks([-0.4,-0.2,0,0.2,0.4])
cbar.ax.tick_params(labelsize=6, length=2, pad=2)

In [ ]:
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
yslice = 128; L = 200; Nh = 1024

ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/5,0,L/10])
ax.set_ylim([-L/5, L/10])
# xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
# ax.plot(xarray,z_mesh[-1,1:,128],lw=0.5,c='k')

# image = plt.pcolormesh(x_mesh1[:,:,yslice], z_mesh1[:,:,yslice], omegax_mesh1[:,:,yslice], shading='flat', 
#                        cmap='RdBu_r', vmax=0.4, vmin=-0.4)
image = plt.pcolormesh(x_mesh2[:,:,yslice], z_mesh2[:,:,yslice], omegax_mesh2[:,:,yslice], shading='flat', 
                       cmap='RdBu_r', vmax=0.4, vmin=-0.4)

cbar = plt.colorbar(image, orientation='vertical')
cbar.ax.text(-0.5, 1.05, r'$\omega_x (s^{-1})$', transform=cbar.ax.transAxes, fontsize=8)
cbar.set_ticks([-0.4,-0.2,0,0.2,0.4])
cbar.ax.tick_params(labelsize=6, length=2, pad=2)

In [ ]:
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
yslice = 128; L = 200; Nh = 1024

ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/5,0,L/10])
ax.set_ylim([-L/5, L/10])
# xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
# ax.plot(xarray,z_mesh[-1,1:,128],lw=0.5,c='k')

# image = plt.pcolormesh(x_mesh1[:,:,yslice], z_mesh1[:,:,yslice], omegax_mesh1[:,:,yslice], shading='flat', 
#                        cmap='RdBu_r', vmax=0.4, vmin=-0.4)
image = plt.pcolormesh(x_mesh2[:,:,yslice], z_mesh2[:,:,yslice], omegaz_mesh2[:,:,yslice], shading='flat', 
                       cmap='RdBu_r', vmax=0.4, vmin=-0.4)

cbar = plt.colorbar(image, orientation='vertical')
cbar.ax.text(-0.5, 1.05, r'$\omega_z (s^{-1})$', transform=cbar.ax.transAxes, fontsize=8)
cbar.set_ticks([-0.4,-0.2,0,0.2,0.4])
cbar.ax.tick_params(labelsize=6, length=2, pad=2)

In [ ]:
dvdx = np.gradient(uy_mesh2, axis=1)/dx
dudy = np.gradient(ux_mesh2, axis=2)/dx 
omega_z = dvdx - dudy

In [ ]:
fig = plt.figure (figsize=[2.5,2]); ax = plt.gca()
yslice = 128; L = 200; Nh = 1024

ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/2,0,L/2])
ax.set_ylim([-L/2, L/2])
image = plt.pcolormesh(x_mesh2[-1,:,:], y_mesh2[-1,:,:], z_mesh2[-1,:-1,:-1], shading='flat', 
                       cmap='RdBu_r', vmax=2, vmin=-2)
# image = plt.pcolormesh(x_mesh2[-1,:,:], y_mesh2[-1,:,:], omegax_mesh2[-1,:,:], shading='flat', 
#                        cmap='RdBu_r', vmax=0.4, vmin=-0.4)
# image = plt.pcolormesh(x_mesh2[-1,:,:], y_mesh2[-1,:,:], omegaz_mesh2[-1,:,:], shading='flat', 
#                        cmap='RdBu_r', vmax=0.4, vmin=-0.4)

cbar = plt.colorbar(image, orientation='vertical')
cbar.ax.text(-0.5, 1.05, r'$\eta (m)$', transform=cbar.ax.transAxes, fontsize=8)
cbar.set_ticks([-2,-1,0,1,2])
# cbar.ax.text(-0.5, 1.05, r'$\omega_x (s^{-1})$', transform=cbar.ax.transAxes, fontsize=8)
# cbar.set_ticks([-0.4,-0.2,0,0.2,0.4])
# cbar.ax.text(-0.5, 1.05, r'$\omega_z (s^{-1})$', transform=cbar.ax.transAxes, fontsize=8)
# cbar.set_ticks([-0.4,-0.2,0,0.2,0.4])

plt.axis('off')
cbar.ax.tick_params(labelsize=6, length=2, pad=2)

In [ ]:
fig = plt.figure(figsize=[2,3])
plt.plot(np.average(omegax_mesh2**2, axis=(1,2)), np.average(z_mesh2, axis=(1,2))[:-1], 'o', mfc='none', ms=3, mew=0.5, label=r'$\langle\omega_x^2\rangle$')
plt.plot(np.average(omegay_mesh2**2, axis=(1,2)), np.average(z_mesh2, axis=(1,2))[:-1], 'o', mfc='none', ms=3, mew=0.5, label=r'$\langle\omega_y^2\rangle$')
plt.plot(np.average(omegaz_mesh2**2, axis=(1,2)), np.average(z_mesh2, axis=(1,2))[:-1], 'o', mfc='none', ms=3, mew=0.5, label=r'$\langle\omega_z^2\rangle$')
plt.ylim([-6,0])
plt.xlim([0,0.1])
plt.xlabel(r'$\langle\omega_i^2\rangle$(s$^{-2}$)')
plt.ylabel(r'$\langle z \rangle$ (m)')
plt.legend(loc='lower right', fancybox=False)

In [ ]:
# TODO: migration to xarray


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=[5,4])
i = 3
# axis 0 is z; axis 1 is x; axis 2 is y (span-wise)
dx = L/Nh
dvdx = np.gradient(uy_mesh_t[i], axis=1)/dx
dudy = np.gradient(ux_mesh_t[i], axis=2)/dx 
omega_z = dvdx - dudy

fig1 = axs[0,0].imshow(omega_z[-1], cmap=plt.get_cmap('coolwarm'), vmax=0.1, vmin=-0.1)
axs[0,0].set_title('$\omega_z | _{z=\eta}$'); axs[0,0].axis('off'); plt.colorbar(fig1, ax=axs[0,0])

fig2 = axs[0,1].imshow(z_mesh_t[i][-1], cmap=plt.get_cmap('coolwarm'), vmax=1.5, vmin=-1.5)
axs[0,1].set_title('$\eta$'); axs[0,1].axis('off'); plt.colorbar(fig2, ax=axs[0,1])

fig3 = axs[1,0].imshow(dvdx[-1], cmap=plt.get_cmap('coolwarm'), vmax=0.4, vmin=-0.4)
axs[1,0].set_title('$\partial v/\partial x | _{z=\eta} $'); axs[1,0].axis('off'); plt.colorbar(fig3, ax=axs[1,0])

fig4 = axs[1,1].imshow(dudy[-1], cmap=plt.get_cmap('coolwarm'), vmax=0.4, vmin=-0.4)
axs[1,1].set_title('$\partial u/\partial y | _{z=\eta} $'); axs[1,1].axis('off'); plt.colorbar(fig4, ax=axs[1,1])




In [ ]:
fig, axs = plt.subplots(1, 5, figsize=[8,2])

itime = 3
# axis 0 is z; axis 1 is x; axis 2 is y (span-wise)
dx = L/Nh
dvdx = np.gradient(uy_mesh_t[itime], axis=1)/dx
dudy = np.gradient(ux_mesh_t[itime], axis=2)/dx 
omega_z = dvdx - dudy

for i in range (0,5):
    contour = axs[i].imshow(omega_z[2+i*3], cmap=plt.get_cmap('coolwarm'), vmax=0.1, vmin=-0.1)
    axs[i].set_title('layer %d' %(3+i*3)); axs[i].axis('off'); 
    
#     plt.colorbar(contour, ax=axs[i])



In [ ]:
for itime in range(0,4):
    epsilon = np.zeros(15)
    z = np.zeros(15)
    dx = L/Nh
    dvdx = np.gradient(uy_mesh_t[itime], axis=1)/dx
    dudy = np.gradient(ux_mesh_t[itime], axis=2)/dx 
    omega_z = dvdx - dudy
    for i in range (0,15):
        epsilon[i] = np.average(omega_z[i]*omega_z[i])
        z[i] = np.average(z_mesh_t[itime][i][:])
    plt.plot(-z,epsilon,label='time %g' %itime)
    plt.plot(-z[:10],(-z[:10])**(-2)*0.1,'--',c='gray')
    plt.plot(-z[8:],(-z[8:])**(-1)*0.02,'--',c='gray')  
    plt.text(1, 0.05, '$z^{-1}$', c='gray')
    plt.text(10, 0.01, '$z^{-2}$', c='gray')
plt.ylim([10**(-8),10**(-1)])    
plt.legend()
plt.xscale('log')
plt.yscale('log')

In [ ]:
for i,t in enumerate((0, 50, 100, 150)):
    ux_mean = np.average(ux_mesh_t[i], axis=(1,2))
    ux_rms = np.average((ux_mesh_t[i] - ux_mean[:,np.newaxis,np.newaxis])**2, axis=(1,2))**0.5
    plt.plot(np.average(z_mesh_t[i][:-1], axis=(1,2)), ux_mean, c=plt.get_cmap('tab20')(i), label='t=%g' %t)
    plt.plot(np.average(z_mesh_t[i][:-1], axis=(1,2)), ux_rms, '--', c=plt.get_cmap('tab20')(i), label='t=%g' %t)
plt.legend()
plt.xlim([-20,0])

In [ ]:
case1 = Case(NL=15, LEVEL=8, path='/projects/DEIKE/jiarongw/multilayer/revision/stokes_ml_9_slope0.577/restart/')
case = case1
# case.L = 200; case.H = 40; case.kp = 2*np.pi/case.L*5; case.cp = (9.8/case.kp)**0.5

def plot_func(t, ax):
    global case
    Nh = 256; Nl = 15; L = 1; H = 1/2
#     cp = case.cp; kp = case.kp
    h_ensem, phi_ensem, ux_ensem, uy_ensem, uz_ensem = read_t_with_phi(t=t, Nh=Nh, Nl=Nl, path=case.path)    
    x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh, phi_mesh = array_to_mesh_with_phi (h_ensem, ux_ensem, uy_ensem, uz_ensem, phi_ensem, L0=L, H=H, Nh=Nh, Nl=Nl)
    yslice = 128
    image = ax.pcolormesh(x_mesh[:,:,yslice], z_mesh[:,:,yslice], ux_mesh[:,:,yslice], shading='flat', cmap='RdBu_r', vmax=0.5, vmin=-0.5)
    ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/2,0,L/2])
    ax.set_ylim([-L/2,L/2])
#     ax.spines.right.set_visible(False)
#     ax.spines.top.set_visible(False)
#     ax.axis('off'); 
#     ax.text(0.05, 0.1, '$t=%gT$' %t, transform=ax.transAxes, fontsize=8)
    ''' Plot the surface '''
    xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
    ax.plot(xarray,np.sum(h_ensem[:,:,128],axis=0)-H,lw=0.5,c='k')
    return image, ux_mesh, uy_mesh, uz_mesh, phi_mesh

t = 2.1
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
image, ux_mesh1, uy_mesh1, uz_mesh1, phi_mesh1 = plot_func(t, ax)

In [ ]:
case1 = Case(NL=15, LEVEL=8, path='/projects/DEIKE/jiarongw/multilayer/revision/stokes_ml_9_slope0.577/restart_from0/')
case = case1
# case.L = 200; case.H = 40; case.kp = 2*np.pi/case.L*5; case.cp = (9.8/case.kp)**0.5

def plot_func(t, ax):
    global case
    Nh = 256; Nl = 15; L = 1; H = 1/2
#     cp = case.cp; kp = case.kp
    h_ensem, phi_ensem, ux_ensem, uy_ensem, uz_ensem = read_t_with_phi(t=t, Nh=Nh, Nl=Nl, path=case.path)    
    x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh, phi_mesh = array_to_mesh_with_phi (h_ensem, ux_ensem, uy_ensem, uz_ensem, phi_ensem, L0=L, H=H, Nh=Nh, Nl=Nl)
    yslice = 128
    image = ax.pcolormesh(x_mesh[:,:,yslice], z_mesh[:,:,yslice], ux_mesh[:,:,yslice], shading='flat', cmap='RdBu_r', vmax=0.5, vmin=-0.5)
    ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/2,0,L/2])
    ax.set_ylim([-L/2,L/2])
#     ax.spines.right.set_visible(False)
#     ax.spines.top.set_visible(False)
#     ax.axis('off'); 
#     ax.text(0.05, 0.1, '$t=%gT$' %t, transform=ax.transAxes, fontsize=8)
    ''' Plot the surface '''
    xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
    ax.plot(xarray,np.sum(h_ensem[:,:,128],axis=0)-H,lw=0.5,c='k')
    return image, ux_mesh, uy_mesh, uz_mesh, phi_mesh

t = 2.1
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
image, ux_mesh2, uy_mesh2, uz_mesh2, phi_mesh2 = plot_func(t, ax)

In [ ]:
case_field1 = Case(NL=15, LEVEL=10, path='/projects/DEIKE/jiarongw/multilayer/revision/field_new_200m_GaussianG0.15S0.5_RE40000_10_15_rand2_Htheta0.503/restart/')
case = case_field1
case.L = 200; case.H = 40; case.kp = 2*np.pi/case.L*5; case.cp = (9.8/case.kp)**0.5

def plot_func(t, ax):
    global case
    Nh = 2**case.LEVEL; Nl = case.NL; L = case.L; H = case.H; cp = case.cp; kp = case.kp
    h_ensem, ux_ensem, uy_ensem, uz_ensem = read_t(t=t, Nh=Nh, Nl=Nl, path=case.path)
    # Additional filtering due to wrong outputs
    h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]
    ux_ensem[0,:,:] = ux_ensem[0,:,:]*0; uy_ensem[0,:,:] = uy_ensem[0,:,:]*0; uz_ensem[0,:,:] = uz_ensem[0,:,:]*0
    
    x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh = array_to_mesh (h_ensem, ux_ensem, uy_ensem, uz_ensem, L0=L, H=H, Nh=Nh, Nl=Nl)
    yslice = 128
    image = ax.pcolormesh(x_mesh[:,:,yslice], z_mesh[:,:,yslice], ux_mesh[:,:,yslice]/cp, shading='flat', cmap='RdBu_r', vmax=0.5, vmin=-0.5)
    ax.set_xticks([-L/2,L/2]); ax.set_yticks([-H,0,L/10])
    ax.set_ylim([-H, L/10])
#     ax.spines.right.set_visible(False)
#     ax.spines.top.set_visible(False)
#     ax.axis('off'); 
#     ax.text(0.05, 0.1, '$t=%gT$' %t, transform=ax.transAxes, fontsize=8)
    ''' Plot the surface '''
    xarray = np.linspace(-L/2,L/2,Nh,endpoint=False) + L/Nh/2
    ax.plot(xarray,np.sum(h_ensem[:,:,128],axis=0)-H,lw=0.5,c='k')
    return image, ux_mesh, uy_mesh, uz_mesh

t = 0
fig = plt.figure (figsize=[4,2]); ax = plt.gca()
image, ux_mesh2, uy_mesh2, uz_mesh2 = plot_func(t, ax)

In [ ]:
fig = plt.figure(figsize=[4,2]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
uy_ensem[0,:,:] = uy_ensem[0,:,:]*0
image = ax.imshow(case_field1.uy_ensem[::-1,30,:]/cp, extent=[-L/2,L/2,-50,0], vmax=0.1, vmin=-0.1, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_x/c_p$')

In [ ]:
case_field1 = Case(NL=15, LEVEL=10, path='/projects/DEIKE/jiarongw/multilayer/field_new_200m_P0.02_RE40000_10_15_rand2_Htheta0.503_N2/')
h_ensem, ux_ensem, uy_ensem, uz_ensem = case_field1.read_t_other(t=100)
h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(case_field1.ux_ensem[::-1,:,100]/cp, extent=[-L/2,L/2,-50,1], vmax=0.2, vmin=-0.2, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_x/c_p$')

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(case_field1.uz_ensem[::-1,:,100]/cp, extent=[-L/2,L/2,-50,1], vmax=0.2, vmin=-0.2, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_z/c_p$')

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(case_field1.uy_ensem[::-1,:,100]/cp, extent=[-L/2,L/2,-50,1], vmax=0.2, vmin=-0.2, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_x/c_p$')

In [ ]:
# axis are z, x, y
ux_ensem[0,:,:] = ux_ensem[0,:,:]*0
uy_ensem[0,:,:] = uy_ensem[0,:,:]*0
uz_ensem[0,:,:] = uz_ensem[0,:,:]*0

z_ensem = uz_ensem*0 # Assembe the vertical coordinate array
NL = 15
for i in range (0,NL):
    z_ensem[i,:,:] = np.sum(h_ensem[:i,:,:], axis=0) + h_ensem[i,:,:]/2

dx = 200/1024; dy = dx
dh = (h_ensem + np.roll(h_ensem,1,axis=0))/2 # last layer is not accurate but it's fine 
dzdx = (z_ensem - np.roll(z_ensem,1,axis=1))/dx # the slope
dzdy = (z_ensem - np.roll(z_ensem,1,axis=2))/dy

duxdz = (ux_ensem - np.roll(ux_ensem,1,axis=0))/dh
duydz = (uy_ensem - np.roll(uy_ensem,1,axis=0))/dh
duzdz = (uz_ensem - np.roll(uz_ensem,1,axis=0))/dh

duzdx = (uz_ensem - np.roll(uz_ensem,1,axis=1))/dx - duzdz*dzdx # what about sign
duzdy = (uz_ensem - np.roll(uz_ensem,1,axis=2))/dy - duzdz*dzdy # what about sign

# duxdx = (ux_ensem - np.roll(ux_ensem,1,axis=1))/dx  # what about sign
duxdx = (ux_ensem - np.roll(ux_ensem,1,axis=1))/dx - duxdz*dzdx # what about sign
# duxdy = (ux_ensem - np.roll(ux_ensem,1,axis=2))/dy  # what about sign
duxdy = (ux_ensem - np.roll(ux_ensem,1,axis=2))/dy - duxdz*dzdy # what about sign


# duydx = (uy_ensem - np.roll(uy_ensem,1,axis=1))/dx
duydx = (uy_ensem - np.roll(uy_ensem,1,axis=1))/dx - duydz*dzdx # what about sign
# duydy = (uy_ensem - np.roll(uy_ensem,1,axis=2))/dy  # what about sign
duydy = (uy_ensem - np.roll(uy_ensem,1,axis=2))/dy - duydz*dzdy # what about sign

In [ ]:
fig = plt.figure(figsize=[4,3]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 

residual = duxdx + duydy + duzdz
image = ax.imshow(np.rot90(residual[-2,:,:]), extent=[-L/2,L/2,-L/2,L/2], vmax=1, vmin=-1, cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/2,L/2])
cbar = plt.colorbar(image)
ax.set_xlabel(r'$x$',labelpad=-2); ax.set_ylabel(r'$y$',labelpad=-2)
cbar.set_label(r'$\nabla\cdot u$')

In [ ]:
omegaz = duydx[1:,:,:] - duxdy[1:,:,:] # ignore bottom layer
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(omegaz[::-1,:,100], extent=[-L/2,L/2,-50,0], vmax=0.2, vmin=-0.2, cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$\Omega_z = \partial u_y/\partial x - \partial u_x/\partial y$')

In [ ]:
omegax = duzdy[1:,:,:] - duydz[1:,:,:] # ignore bottom layer
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(omegax[::-1,:,100], extent=[-L/2,L/2,-50,0], vmax=0.2, vmin=-0.2, cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$\Omega_x = \partial u_z/\partial y - \partial u_y/\partial z$')
# cbar.set_label(r'$u_x/c_p$')

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(ux_ensem[::-1,:,100]/cp, extent=[-L/2,L/2,-50,0], vmax=0.2, vmin=-0.2, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_x/c_p$')

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(duydx[::-1,:,100], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(duxdy[::-1,:,100], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)

In [ ]:
omegax = duzdy[1:,:,:] - duydz[1:,:,:] # ignore bottom layer
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
slopeterm = duzdz*dzdx
image = ax.imshow(slopeterm[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)

In [ ]:
omegax = duzdy[1:,:,:] + duydz[1:,:,:] # ignore bottom layer
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], vmax=1, vmin=-1, cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(duydz[::-1,100,:], extent=[-L/2,L/2,-50,0], vmax=1, vmin=-1, cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)

In [ ]:
omegax = duzdy[1:,:,:] - duydz[1:,:,:] # ignore bottom layer
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(omegax[::-1,100,:], extent=[-L/2,L/2,-50,0], vmax=1, vmin=-1, cmap='RdBu', alpha=0.9)
# image = ax.imshow(duzdy[::-1,100,:], extent=[-L/2,L/2,-50,0], cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)

In [ ]:
fig = plt.figure(figsize=[4,2]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(duydz[::-1,100,:], extent=[-L/2,L/2,-50,0], vmax=1, vmin=-1, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])
cbar = plt.colorbar(image)
# cbar.set_label(r'$u_x/c_p$')

In [ ]:
# How to speed up matplotlib
# https://stackoverflow.com/questions/8955869/why-is-plotting-with-matplotlib-so-slow
from matplotlib import animation
from visualization import contour
from IPython.display import HTML
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import matplotlib.image as mpimg

# Single animation generation function
def plot_animation(animate_function, frame_number = 31, interval_time = 100):

    # First set up the figure, the axis, and the plot element we want to animate   
    global case
    fig = plt.figure(figsize=[4,2]); ax = plt.gca()

    # animation function.  This is called sequentially
    def animate(i):
        imgplot = animate_function(i, ax)
        return imgplot,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, frames=frame_number, interval=interval_time, blit = True)  
    return anim


# Define the function called at every animation time to read in images
def plot_func(i, ax):
    global case
    L = 200; kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5; Nh = 1024; Nl = 15
    t = 110+i*0.2
    h_ensem, ux_ensem, uy_ensem, uz_ensem = case.read_t_other(t=t)
    h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]
    ax.clear()
    x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh = array_to_mesh (h_ensem, ux_ensem, uy_ensem, uz_ensem, L0=200, H=40, Nh=1024, Nl=15)
    ux_mesh[0,:,:] = ux_mesh[0,:,:]*0
    uy_mesh[0,:,:] = uy_mesh[0,:,:]*0
    uz_mesh[0,:,:] = uz_mesh[0,:,:]*0
    xslice = 669
    yslice = 758
    image = ax.pcolormesh(x_mesh[:,:,yslice], z_mesh[:,:,yslice], uy_mesh[:,:,yslice]/cp, shading='flat', cmap='RdBu_r', vmax=0.2, vmin=-0.2)
    ax.set_xticks([-L/2,L/2]); ax.set_yticks([-40,0])
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    print(i)
    return image

case = Case(NL=15, LEVEL=10, path='/projects/DEIKE/jiarongw/multilayer/field_new_200m_P0.02_RE40000_10_15_rand2_Htheta0.503/')

anim = plot_animation(plot_func, frame_number = 20, interval_time = 100) # Specify frame number
HTML(anim.to_html5_video())

In [ ]:
""" Assemble the 3D array of vertice for pcolormesh. """

def array_to_mesh (h_ensem, ux_ensem, uy_ensem, uz_ensem, L0=200, H=40, Nh=512, Nl=15):
    fieldnames = ['x', 'y', 'z', 'ux', 'uy', 'uz', 'f']
 
    x_mesh = np.zeros([Nl+1,Nh+1,Nh+1]) # Different from the vtk format
    y_mesh = np.zeros([Nl+1,Nh+1,Nh+1])
    z_mesh = np.zeros([Nl+1,Nh+1,Nh+1])
    f_mesh = np.zeros([Nl,Nh,Nh])
    ux_mesh = np.zeros([Nl,Nh,Nh])
    uy_mesh = np.zeros([Nl,Nh,Nh])
    uz_mesh = np.zeros([Nl,Nh,Nh])

    h_ensem_expand = np.zeros([Nl,Nh+1,Nh+1]) # Need to go from centered to grid, pad the array
    h_ensem_expand[:,:Nh,:Nh] = np.copy(h_ensem) # Need to go from centered to grid
    h_ensem_expand[:,Nh,:Nh] = np.copy(h_ensem[:,Nh-1,:Nh])
    h_ensem_expand[:,:Nh,Nh] = np.copy(h_ensem[:,:Nh,Nh-1])
    h_ensem_expand[:,Nh,Nh] = np.copy(h_ensem[:,Nh-1,Nh-1])
    h_ensem_expand = np.array(h_ensem_expand)

    xarray = np.linspace(-L0/2, L0/2, Nh+1, endpoint=True)
    yarray = np.linspace(-L0/2, L0/2, Nh+1, endpoint=True)

    for k in range(Nl+1):
        for i in range(Nh+1):
            for j in range(Nh+1):
                z_mesh[k,i,j] = np.sum(h_ensem_expand[:k,i,j]) - H
                x_mesh[k,i,j] = xarray[i]
                y_mesh[k,i,j] = yarray[j]

    for k in range(Nl):
        for i in range(Nh):
            for j in range(Nh):
                ux_mesh[k,i,j] = ux_ensem[k,i,j]
                uy_mesh[k,i,j] = uy_ensem[k,i,j]
                uz_mesh[k,i,j] = uz_ensem[k,i,j]
                if k == Nl-1: # surface layer
                    f_mesh[k,i,j] = 0
                else:
                    f_mesh[k,i,j] = 1

    return x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh

In [ ]:
""" Ridge detection """
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
def detect_ridges(gray, sigma=1):
    H_elems = hessian_matrix(gray, sigma=sigma, order='rc')
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    return maxima_ridges, minima_ridges

In [ ]:
fig = plt.figure(figsize=[2,2]); ax = plt.gca()
eta = z_mesh[14,:,:]
L = 200; Nh = 1024; kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5
threshold = -3*kp

# image = ax.imshow(eta, cmap='gray', alpha=1, vmax=1, vmin=-1, extent=[0,L,0,L])
# # image = ax.imshow(ux[256:,256:]/case.config.cp, vmax=-0.2, vmin=0.6, cmap='RdBu_r', alpha=1, extent=[0,case.config.L0/2,-case.config.L0/2,0])
# cbar = plt.colorbar(image, shrink=0.5, pad=0.05)

image = ax.imshow(ux_mesh[13,:,:]/cp, vmax=-1, vmin=1, cmap='RdBu_r', alpha=1, extent=[0,L,0,L])
cbar = plt.colorbar(image, shrink=0.5, pad=0.05)
# image = ax.imshow(a_, cmap='Reds', interpolation='none', clim=[0.5,1.2], alpha=1, extent=[0,L,0,L])

a, b = detect_ridges(eta, sigma=1.0) # Maxima and minima ridges
delta = L/Nh # Normalize the curvature by grid size
b_norm = b/delta**2                
b_ = np.logical_not(b_norm > threshold) # Is this value fixed???
height_filter = np.logical_not(eta < 2.5*np.var(eta)**0.5)
b_ = b_*height_filter
a_ = np.zeros(b_.shape) 
for i in range(0,Nh-1):
    for j in range(1,Nh-1):
        if (b_[i][j-1] > 0) and (b_[i][j+1] == 0):
            a_[i][j] = 1
#             print (i,j)

In [ ]:
fig = plt.figure(figsize=[2,2]); ax = plt.gca()
eta = z_mesh[14,:,:]
L = 200; Nh = 1024; kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5
threshold = -3*kp

# image = ax.imshow(eta, cmap='gray', alpha=1, vmax=1, vmin=-1, extent=[0,L,0,L])
# # image = ax.imshow(ux[256:,256:]/case.config.cp, vmax=-0.2, vmin=0.6, cmap='RdBu_r', alpha=1, extent=[0,case.config.L0/2,-case.config.L0/2,0])
# cbar = plt.colorbar(image, shrink=0.5, pad=0.05)

image = ax.imshow(uz_mesh[13,:,:]/cp, vmax=-0.5, vmin=0.5, cmap='RdBu_r', alpha=1, extent=[0,L,0,L])
cbar = plt.colorbar(image, shrink=0.5, pad=0.05)
# image = ax.imshow(a_, cmap='Reds', interpolation='none', clim=[0.5,1.2], alpha=1, extent=[0,L,0,L])

a, b = detect_ridges(eta, sigma=1.0) # Maxima and minima ridges
delta = L/Nh # Normalize the curvature by grid size
b_norm = b/delta**2                
b_ = np.logical_not(b_norm > threshold) # Is this value fixed???
height_filter = np.logical_not(eta < 2.5*np.var(eta)**0.5)
b_ = b_*height_filter
a_ = np.zeros(b_.shape) 
for i in range(0,Nh-1):
    for j in range(1,Nh-1):
        if (b_[i][j-1] > 0) and (b_[i][j+1] == 0):
            a_[i][j] = 1
#             print (i,j)

In [ ]:
x_mesh, y_mesh, z_mesh, ux_mesh, uy_mesh, uz_mesh = array_to_mesh (h_ensem, ux_ensem, uy_ensem, uz_ensem, L0=200, H=40, Nh=1024, Nl=15)
ux_mesh[0,:,:] = ux_mesh[0,:,:]*0
uy_mesh[0,:,:] = uy_mesh[0,:,:]*0
uz_mesh[0,:,:] = uz_mesh[0,:,:]*0
L = 200; kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5

In [ ]:
fig = plt.figure(figsize=[4,1]); ax = plt.gca()
xslice = 669
yslice = 758
plt.pcolormesh(x_mesh[:,:,yslice], z_mesh[:,:,yslice], uy_mesh[:,:,yslice]/cp, shading='flat', cmap='RdBu_r', vmax=0.3, vmin=-0.3)
plt.axvline(xslice/Nh*L-L/2, lw=0.5, c='gray')
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

In [ ]:
fig = plt.figure(figsize=[4,1]); ax = plt.gca()
ux_mesh[0,:,:] = ux_mesh[0,:,:]*0
uy_mesh[0,:,:] = uy_mesh[0,:,:]*0
uz_mesh[0,:,:] = uz_mesh[0,:,:]*0
L = 200; kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5
xslice = 669
yslice = 758
plt.pcolormesh(y_mesh[:,xslice,:], z_mesh[:,xslice,:], uz_mesh[:,yslice,:]/cp, shading='flat', cmap='RdBu_r', vmax=0.3, vmin=-0.3)
plt.axvline(yslice/Nh*L-L/2, lw=0.5, c='gray')
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

In [ ]:
fig = plt.figure(figsize=[5,4]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 

image = ax.imshow(np.rot90(case_field1.ux_ensem[-1,:,:])/cp, extent=[-L/2,L/2,-L/2,L/2], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)
# image = ax.imshow(case_field1.ux_ensem[-1,:,:]/cp, extent=[-L/2,L/2,-L/2,L/2], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)

ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/2,L/2])

v1 = np.linspace(-0.4, 0.4, 3, endpoint=True)
cbar = fig.colorbar(image, orientation="vertical", ticks=v1)

# x_ = np.linspace(-100,100,100)
# y_ = np.ones(100)*(220/Nh)*200-100
# plt.plot(x_, y_, c='gray')
x_ = np.linspace(-100,100,100)
y_ = np.ones(100)*(85/Nh)*200-100
plt.plot(x_, y_, '--', lw=1, c='gray')

cbar.set_label(r'$u_x/c_p$')
plt.xlabel('$x(m)$', labelpad=-5)
plt.ylabel('$z(m)$', labelpad=-10)
fig.savefig('figures_backup/bulk_velocity_topdown_pof.pdf', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=[4,2]); ax = plt.gca()
x_ = np.linspace(-100,100,1024)
sl = 85
h_bottom = case.h_ensem[1,:,sl]*case.h_ensem[1,:,sl]/case.h_ensem[2,:,sl]
# eta = np.sum(case_field1.h_ensem[:,:,sl],axis=0)+h_bottom-40
# plt.plot(x_, eta)

plt.plot(x_, np.sum(case_field1.h_ensem[1:-1,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-2,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-3,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-4,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-5,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-6,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-7,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-8,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-9,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-10,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-11,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-12,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-13,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-Nl-1,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-Nl,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, h_bottom-40, lw=0.5, c='k')
plt.plot(x_, h_bottom*0-40, lw=0.5, c='k')
plt.ylim([-54,10])
plt.xlim([-105,130])


ax.fill_between(x_, h_bottom*0-50, h_bottom*0-40, color='gray', alpha=0.9)

# Move the left and bottom spines to x = 0 and y = 0, respectively.
# ax.spines[["left", "bottom"]].set_position(("data", 0))
# Hide the top and right spines.
ax.spines[["top", "right"]].set_visible(False)

# Draw arrows (as black triangles: ">k"/"^k") at the end of the axes.  In each
# case, one of the coordinates (0) is a data coordinate (i.e., y = 0 or x = 0,
# respectively) and the other one (1) is an axes coordinate (i.e., at the very
# right/top of the axes).  Also, disable clipping (clip_on=False) as the marker
# actually spills out of the axes.
ax.plot(1, 0, ">k", transform=ax.transAxes, clip_on=False, markersize=2)
ax.plot(0, 1, "^k", transform=ax.transAxes, clip_on=False, markersize=2)
# ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
# ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
ax.text(1.02, -0.02, '$x$', transform=ax.transAxes, clip_on=False)
ax.text(-0.01, 1.05, '$z$', transform=ax.transAxes, clip_on=False)
ax.text(0.3, 0.12, 'Bottom topology', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.83, r'$\eta(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.2, r'$z_{b}(\mathbf{x})$', transform=ax.transAxes, clip_on=False, fontsize=8)

ax.text(0.Nl, 0.27, r'$h_0(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.28, 0.27, r'$\mathbf{u_{0}}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.41, 0.27, r'$w_{0}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.55, 0.27, r'$p_{nh0}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.Nl, 0.4, r'$h_1(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.28, 0.4, r'$\mathbf{u_{1}}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.41, 0.4, r'$w_{1}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.55, 0.4, r'$p_{nh1}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.Nl, 0.5, r'$h_2(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.28, 0.5, r'$\mathbf{u_{2}}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.41, 0.5, r'$w_{2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.55, 0.5, r'$p_{nh2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.Nl, 0.6, r'...', transform=ax.transAxes, clip_on=False, fontsize=8)

ax.text(0.9, 0.55, r'$z_{5/2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.45, r'$z_{3/2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.35, r'$z_{1/2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)


ax.set_xticks([]); ax.set_yticks([])


In [ ]:
fig = plt.figure(figsize=[4,3]); ax = plt.gca()
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
L = 1
image = ax.imshow(np.rot90(ux_ensem[19])/cp, extent=[-L/2,L/2,-L/2,L/2], vmax=1, vmin=-0.2, cmap='viridis', alpha=0.9)
ax.set_xticks([-L/2,0,L/2]); ax.set_yticks([-L/2,0,L/2])
cbar = plt.colorbar(image)
# nx = 4Nl; ny = 30
# ax.plot((nx-N/2)/N*L,(ny-N/2)/N*L,marker='o',markersize=1,c='r')
cbar.set_label(r'$u_x/c_p$')
plt.xlabel('$x$(m)')
plt.ylabel('$y$(m)', labelpad=-10)
# fig.savefig('figures/ux_500m.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig = plt.figure(figsize=[4,2]); ax = plt.gca()
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
L = 1
image = ax.imshow(case1.ux_ensem[::-1,:,30]/cp, extent=[-L/2,L/2,-0.5,0], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-0.5,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_x/c_p$')

In [ ]:
fig = plt.figure(figsize=[4,2]); ax = plt.gca()
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(case2.ux_ensem[::-1,:,30]/cp, extent=[-L/2,L/2,-0.5,0], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-0.5,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_x/c_p$')

In [ ]:
fig = plt.figure(figsize=[4,2]); ax = plt.gca()
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 
image = ax.imshow(case3.uz_ensem[::-1,:,30]/cp, extent=[-L/2,L/2,-0.5,0], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-0.5,0])
cbar = plt.colorbar(image)
cbar.set_label(r'$u_x/c_p$')

In [ ]:
fig = plt.figure(figsize=[5,4]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 

image = ax.imshow(np.rot90(case_field1.ux_ensem[-1,:,:])/cp, extent=[-L/2,L/2,-L/2,L/2], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)
# image = ax.imshow(case_field1.ux_ensem[-1,:,:]/cp, extent=[-L/2,L/2,-L/2,L/2], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)

ax.set_xticks([-L/2,L/2]); ax.set_yticks([-L/2,L/2])

v1 = np.linspace(-0.4, 0.4, 3, endpoint=True)
cbar = fig.colorbar(image, orientation="vertical", ticks=v1)

# x_ = np.linspace(-100,100,100)
# y_ = np.ones(100)*(220/Nh)*200-100
# plt.plot(x_, y_, c='gray')
x_ = np.linspace(-100,100,100)
y_ = np.ones(100)*(85/Nh)*200-100
plt.plot(x_, y_, '--', lw=1, c='gray')

cbar.set_label(r'$u_x/c_p$')
plt.xlabel('$x(m)$', labelpad=-5)
plt.ylabel('$z(m)$', labelpad=-10)
fig.savefig('figures_backup/bulk_velocity_topdown_pof.pdf', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=[5,1]); ax = plt.gca()
L = 200
kp = 2*np.pi/L*5; cp = (9.8/kp)**0.5 

case_field1.ux_ensem[0,:,:] = case_field1.ux_ensem[0,:,:]*0 # THIS IS NOT GOOD BUT WE REMOVED THE BOTTOM LINE ANOMALY
image = ax.imshow(case_field1.ux_ensem[::-1,:,85]/cp, extent=[-L/2,L/2,-50,0], vmax=0.4, vmin=-0.4, cmap='RdBu', alpha=0.9)
ax.set_xticks([-L/2,L/2]); ax.set_yticks([-50,0])

v1 = np.linspace(-0.4, 0.4, 3, endpoint=True)
cbar = fig.colorbar(image, orientation="vertical", ticks=v1)

cbar.set_label(r'$u_x/c_p$')
plt.xlabel('$x(m)$', labelpad=-5)
plt.ylabel('$z(m)$', labelpad=-10)
fig.savefig('figures_backup/bulk_velocity_pof.pdf', bbox_inches='tight')

### <a class="anchor" id="1">Draw an illustration of the layers</a>

In [ ]:
case_field1 = Case(NL=Nl, LEVEL=9, path='/projects/DEIKE/jiarongw/multilayer/field_new_200m_P0.03_RE40000_9_Nl_rand2_Htheta0.503/')
h_ensem, ux_ensem, uy_ensem, uz_ensem = case_field1.read_t(t=120)

In [ ]:
fig = plt.figure(figsize=[4,2]); ax = plt.gca()
x_ = np.linspace(-100,100,Nh)
sl = 85
h_bottom = case_field1.h_ensem[1,:,sl]*case_field1.h_ensem[1,:,sl]/case_field1.h_ensem[2,:,sl]
# eta = np.sum(case_field1.h_ensem[:,:,sl],axis=0)+h_bottom-40
# plt.plot(x_, eta)

plt.plot(x_, np.sum(case_field1.h_ensem[1:-1,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-2,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-3,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-4,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-5,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-6,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-7,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-8,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-9,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-10,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-11,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-12,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-13,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-Nl-1,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-Nl,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, h_bottom-40, lw=0.5, c='k')
plt.plot(x_, h_bottom*0-40, lw=0.5, c='k')
plt.ylim([-54,10])
plt.xlim([-105,130])


ax.fill_between(x_, h_bottom*0-50, h_bottom*0-40, color='gray', alpha=0.9)

# Move the left and bottom spines to x = 0 and y = 0, respectively.
# ax.spines[["left", "bottom"]].set_position(("data", 0))
# Hide the top and right spines.
ax.spines[["top", "right"]].set_visible(False)

# Draw arrows (as black triangles: ">k"/"^k") at the end of the axes.  In each
# case, one of the coordinates (0) is a data coordinate (i.e., y = 0 or x = 0,
# respectively) and the other one (1) is an axes coordinate (i.e., at the very
# right/top of the axes).  Also, disable clipping (clip_on=False) as the marker
# actually spills out of the axes.
ax.plot(1, 0, ">k", transform=ax.transAxes, clip_on=False, markersize=2)
ax.plot(0, 1, "^k", transform=ax.transAxes, clip_on=False, markersize=2)
# ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
# ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
ax.text(1.02, -0.02, '$x$', transform=ax.transAxes, clip_on=False)
ax.text(-0.01, 1.05, '$z$', transform=ax.transAxes, clip_on=False)
ax.text(0.3, 0.12, 'Bottom topology', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.83, r'$\eta(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.2, r'$z_{b}(\mathbf{x})$', transform=ax.transAxes, clip_on=False, fontsize=8)

ax.text(0.Nl, 0.27, r'$h_0(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.28, 0.27, r'$\mathbf{u_{0}}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.41, 0.27, r'$w_{0}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.55, 0.27, r'$p_{nh0}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.Nl, 0.4, r'$h_1(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.28, 0.4, r'$\mathbf{u_{1}}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.41, 0.4, r'$w_{1}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.55, 0.4, r'$p_{nh1}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.Nl, 0.5, r'$h_2(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.28, 0.5, r'$\mathbf{u_{2}}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.41, 0.5, r'$w_{2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.55, 0.5, r'$p_{nh2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.Nl, 0.6, r'...', transform=ax.transAxes, clip_on=False, fontsize=8)

ax.text(0.9, 0.55, r'$z_{5/2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.45, r'$z_{3/2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)
ax.text(0.9, 0.35, r'$z_{1/2}(\mathbf{x},t)$', transform=ax.transAxes, clip_on=False, fontsize=8)


ax.set_xticks([]); ax.set_yticks([])

fig.savefig('figures/sketch_ml.pdf', bbox_inches='tight')
fig.savefig('figures_vector/sketch_ml.eps', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=[2,3])
case = case_field1
# h_aver = np.average(case.h_ensem, axis=(1,2))
h_aver = case.h_ensem[:,85,85]
h_aver[0] = h_aver[1]*h_aver[1]/h_aver[2] # Because of the first layer not detected...
z_aver = np.zeros(len(h_aver))
for i in range(len(z_aver)):
    z_aver[i] = np.sum(h_aver[0:i+1]) 

plt.plot(z_aver-40,'.',markersize=3)

h_aver = case.h_ensem[:,400,85]
h_aver[0] = h_aver[1]*h_aver[1]/h_aver[2] # Because of the first layer not detected...
z_aver = np.zeros(len(h_aver))
for i in range(len(z_aver)):
    z_aver[i] = np.sum(h_aver[0:i+1]) 
plt.plot(z_aver-40,'.',markersize=3)

# plt.plot(np.var(h_ensem, axis=(1,2))**0.5,'.')
# plt.plot(np.average(ux_ensem, axis=(1,2)),'.')
# plt.xlim([0,30])
plt.ylim([-40,0])
plt.xlabel('# of layer')
plt.ylabel(r'$z$')

In [ ]:
h_bottom = case_field1.h_ensem[1,:,sl]*case_field1.h_ensem[1,:,sl]/case_field1.h_ensem[2,:,sl]
# eta = np.sum(case_field1.h_ensem[:,:,sl],axis=0)+h_bottom-40
# plt.plot(x_, eta)

plt.plot(x_, np.sum(case_field1.h_ensem[1:-1,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-2,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-3,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-4,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-5,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-6,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-7,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-8,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-9,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-10,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-11,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-12,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-13,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-Nl-1,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, np.sum(case_field1.h_ensem[1:-Nl,:,sl],axis=0)+h_bottom-40, lw=0.5, c='k')
plt.plot(x_, h_bottom-40, lw=0.5, c='k')
plt.plot(x_, h_bottom*0-40, lw=0.5, c='k')
plt.ylim([-54,10])
plt.xlim([-105,130])

In [ ]:
case_field1.h_ensem[0,:,:] = case_field1.h_ensem[1,:,:]*case_field1.h_ensem[1,:,:]/case_field1.h_ensem[2,:,:]
h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]
# field_new_200m_P0.03_RE40000_9_Nl_rand2_Htheta0.503
case_field1.L0 = 200 # Box size
case_field1.H = 40 # Depth

In [ ]:
""" Writing to csv """
import csv
file = open("multilayer.csv", "w")
fieldnames = ['x', 'y', 'z', 'ux', 'uy', 'uz', 'f']
writer = csv.writer(file)
writer.writerow(fieldnames) 

z_ensem = h_ensem*0
f_ensem = h_ensem*0

case = case_field1

xarray = np.linspace(-case.L0/2, case.L0/2, Nh)
yarray = np.linspace(-case.L0/2, case.L0/2, Nh)

for k in range(Nl):
    for i in range(Nh):
        for j in range(Nh):
            z_ensem[k,i,j] = np.sum(h_ensem[:k,i,j]) + 0.5*h_ensem[k,i,j] - case.H
            if k == Nl-1: # surface layer
                f_ensem[k,i,j] = 0
            else:
                f_ensem[k,i,j] = 1
            writer.writerow([xarray[i], yarray[j], z_ensem[k,i,j], ux_ensem[k,i,j], uy_ensem[k,i,j], uz_ensem[k,i,j], f_ensem[k,i,j]])     

file.close()

### Output to Paraview
Reading from the directory of `/projects/DEIKE/jiarongw/multilayer/paraview/field/` and there are about 200 frames 0.2 apart from 100 to Nl-10

In [ ]:
from pyevtk.hl import gridToVTK
from tqdm import tqdm

In [ ]:
def read (pre='eta_matrix_', t=100, index=0, N=Nh, filepath='/projects/DEIKE/jiarongw/multilayer/paraview/'):
    filename = filepath + pre + 't%g_' %t + 'l%g' %index
    f = np.fromfile(filename, dtype=np.float32)
    f = f.reshape(N+1,N+1); f = f[1:,1:]
    return f

def read_t (t=100, filepath='/projects/DEIKE/jiarongw/multilayer/paraview/', N=Nh):
    h_ensem = []; ux_ensem = []; uy_ensem = []; uz_ensem = []
    for l in range (0,Nl):
        h = read(pre='h_matrix_', t=t, index=l, N=N, filepath=filepath)
        ux = read(pre='ux_matrix_', t=t, index=l, N=N, filepath=filepath)
        uy = read(pre='uy_matrix_', t=t, index=l, N=N, filepath=filepath)
        uz = read(pre='uz_matrix_', t=t, index=l, N=N, filepath=filepath)
        h_ensem.append(h)
        ux_ensem.append(ux)
        uy_ensem.append(uy)
        uz_ensem.append(uz)

    h_ensem = np.array (h_ensem)
    ux_ensem = np.array (ux_ensem)
    uy_ensem = np.array (uy_ensem)
    uz_ensem = np.array (uz_ensem)
    
    return (h_ensem, ux_ensem, uy_ensem, uz_ensem)

In [ ]:
""" convert the 3D array to vtk file for paraview. Need to specify L0 and H """

def array_to_vtk (h_ensem, ux_ensem, uy_ensem, uz_ensem, ichoice, filepath='/projects/DEIKE/jiarongw/multilayer/paraview/vtk/', L0=200, H=40):
    fieldnames = ['x', 'y', 'z', 'ux', 'uy', 'uz', 'f']

    x_vtk = np.zeros([Nh+1,Nh+1,Nl+1])
    y_vtk = np.zeros([Nh+1,Nh+1,Nl+1])
    z_vtk = np.zeros([Nh+1,Nh+1,Nl+1])
    f_vtk = np.zeros([Nh,Nh,Nl])
    ux_vtk = np.zeros([Nh,Nh,Nl])
    uy_vtk = np.zeros([Nh,Nh,Nl])
    uz_vtk = np.zeros([Nh,Nh,Nl])

    h_ensem_expand = np.zeros([Nl,Nh+1,Nh+1]) # Need to go from centered to grid, pad the array
    h_ensem_expand[:,:Nh,:Nh] = np.copy(h_ensem) # Need to go from centered to grid
    h_ensem_expand[:,Nh,:Nh] = np.copy(h_ensem[:,Nh-1,:Nh])
    h_ensem_expand[:,:Nh,Nh] = np.copy(h_ensem[:,:Nh,Nh-1])
    h_ensem_expand[:,Nh,Nh] = np.copy(h_ensem[:,Nh-1,Nh-1])
    h_ensem_expand = np.array(h_ensem_expand)

    xarray = np.linspace(-L0/2, L0/2, Nh+1, endpoint=True)
    yarray = np.linspace(-L0/2, L0/2, Nh+1, endpoint=True)

    for k in range(Nl+1):
        for i in range(Nh+1):
            for j in range(Nh+1):
                z_vtk[i,j,k] = np.sum(h_ensem_expand[:k,i,j]) - H
                x_vtk[i,j,k] = xarray[i]
                y_vtk[i,j,k] = yarray[j]

    for k in range(Nl):
        for i in range(Nh):
            for j in range(Nh):
                ux_vtk[i,j,k] = ux_ensem[k,i,j]
                uy_vtk[i,j,k] = uy_ensem[k,i,j]
                uz_vtk[i,j,k] = uz_ensem[k,i,j]
                if k == Nl-1: # surface layer
                    f_vtk[i,j,k] = 0
                else:
                    f_vtk[i,j,k] = 1
                    
    gridToVTK(filepath + "structured_%g" %ichoice, x_vtk, y_vtk, z_vtk, cellData = {"f": f_vtk, "ux": ux_vtk, "uy": uy_vtk, "uz": uz_vtk})
    return x_vtk, y_vtk, z_vtk

In [ ]:
for i in tqdm(range(0,100)):
    tchoice = 100 + i*0.2
    h_ensem, ux_ensem, uy_ensem, uz_ensem = read_t(t=tchoice, filepath='/projects/DEIKE/jiarongw/multilayer/paraview/field/', N=Nh)
    h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]
    ux_ensem[0,:,:] = ux_ensem[0,:,:]*0; uy_ensem[0,:,:] = uy_ensem[0,:,:]*0; uz_ensem[0,:,:] = uz_ensem[0,:,:]*0
    x_vtk, y_vtk, z_vtk = array_to_vtk (h_ensem, ux_ensem, uy_ensem, uz_ensem, ichoice=i, filepath='/projects/DEIKE/jiarongw/multilayer/paraview/vtk/')
    

In [ ]:
ux_ensem.min()

In [ ]:
img = plt.imshow(ux_ensem[13,:,:])
plt.colorbar(img)

In [ ]:
# img = plt.imshow(h_ensem[0,:,:] - 40)
# plt.colorbar(img)

h_ensem[0,:,:] = h_ensem[1,:,:]*h_ensem[1,:,:]/h_ensem[2,:,:]


In [ ]:
h_ensem.shape